In [4]:
import nilmtk

start_time = '2015-01-01'
end_time = '2015-04-01'
sample_period = 10


ukdale = nilmtk.DataSet("ukdale2.h5")
ukdale.set_window(start=start_time, end=end_time)


arr = []
for j in range(len(ukdale.buildings)):
    arr2 = []
    for i in range(len(ukdale.buildings[j+1].elec.appliances)):
        arr2.append(ukdale.buildings[1].elec.appliances[i].type["type"])
    arr.append(arr2)

common_apps = set(arr[0]).intersection(arr[1], arr[4])
print(common_apps)

{'hair straighteners', 'toaster', 'light', 'wireless phone charger', 'audio system', 'mobile phone charger', 'broadband router', 'fan', 'radio', 'food processor', 'drill', 'computer monitor', 'USB hub'}


In [72]:

for i in ukdale.buildings[1].elec.meters:
    print(len(i.appliances))

1
1
2
2
1
1
2
1
3
3
1
1
1
1
1
1
2
2
1
2
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
3
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
6


In [89]:

single_meters = []

# ukdale.buildings[1].elec.instance()
# num_elec = max(ukdale.buildings[1].elec.)
buildings = [1,2,5]
metersPerBuilding = [53,19,5,6,25]
all_meters = []

for i in buildings:
    # metersPerBuilding.append(len(ukdale.buildings[i+1].elec.meters))
    for j in ukdale.buildings[1].elec.meters:
        if len(j.appliances) > 1:
            print(j.instance())




# print(metersPerBuilding)

# ukdale.buildings[1].elec[18].appliances

# i = 0
# for i in ukdale.buildings[1].elec[18].appliances:
#     print(elec.submeter())

TypeError: 'MeterGroup' object is not an iterator

In [93]:
df_microwave =next(ukdale.buildings[1].elec.meters.load())
df_microwave

Loading data for meter ElecMeterID(instance=54, building=1, dataset='UK-DALE')     

c:\users\stefa\onedrive - fhwn\privat\studium\mit_2-semester\case_study\smart_meter_predictions\nilmtk\nilmtk\utils.py:509: UserWarning: Found duplicate index. Keeping first value
  warnings.warn("Found duplicate index. Keeping first value")



Done loading data all meters for this chunk.


physical_quantity               power                  voltage
type                         apparent       active            
2015-01-01 00:00:00+00:00  235.380005   201.301666  244.543335
2015-01-01 00:00:06+00:00  237.518341   201.231659  244.259995
2015-01-01 00:00:12+00:00  235.695007   200.365005  244.485001
2015-01-01 00:00:18+00:00  235.523331   200.134995  244.139999
2015-01-01 00:00:24+00:00  237.291672   199.263336  244.388336
...                               ...          ...         ...
2015-03-31 23:59:30+01:00  677.593323  1066.051636  244.603333
2015-03-31 23:59:36+01:00  677.836670  1065.925049  244.661667
2015-03-31 23:59:42+01:00  676.351685  1068.528320  244.803329
2015-03-31 23:59:48+01:00  675.723328  1064.956665  244.696655
2015-03-31 23:59:54+01:00  677.961670  1065.496704  244.681671

[1295400 rows x 3 columns]

In [ ]:
from nilmtk.api import API
from nilmtk.disaggregate import Mean
from nilmtk_contrib.disaggregate import DAE,Seq2Point, Seq2Seq, RNN

epochs = 1

redd = {
    'power': {
        'mains': ['apparent','active'],
        'appliance': ['apparent','active']
    },
    'sample_rate': 10,

    'appliances': ['microwave', 'dish washer', 'washer dryer', 'coffee maker'],
    'methods': {
        'RNN':RNN({'n_epochs':epochs,'batch_size':32}),
        'DAE':DAE({'n_epochs':epochs,'batch_size':32}),
        'Seq2Point':Seq2Point({'n_epochs':epochs,'batch_size':32}),
        'Seq2Seq':Seq2Seq({'n_epochs':epochs,'batch_size':32}),
        'Mean': Mean({}),

    },
    'train': {
        'datasets': {
            'Dataport': {
                'path': 'C:/Users/david/Desktop/smart_meter_predictions/ukdale2.h5',
                'buildings': {
                    1: {
                        'start_time': '2015-04-04',
                        'end_time': '2015-04-05'
                    },
                    2: {
                        'start_time': '2015-01-28',
                        'end_time': '2015-01-30'
                        },
                }

            }
        }
    },
    'test': {
        'datasets': {
            'Datport': {
                'path': 'C:/Users/david/Desktop/smart_meter_predictions/ukdale2.h5',
                'buildings': {
                    1: {
                        'start_time': '2015-04-05',
                        'end_time': '2015-04-06'
                    },
                }
            }
        },
        'metrics':['mae']
    }
}

#'WindowGRU':WindowGRU({'n_epochs':50,'batch_size':32}),